# Exploratory Data Analysis

The goal of this notebook is to look for general facts about our data.

In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression

from google.colab import drive
drive.mount('/content/drive')
drive_dir = '/content/drive/My Drive/Project_3/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv(drive_dir + 'data/promotion_train_data.csv')

#Cleaning Data

In [ ]:
k = df[df['KPIs_met>80%'] == 1]
k[k['awards_won?'] == 1]

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met>80%,awards_won?,avg_training_score,is_promoted,starting_age,total_training_score,age_rating,age_KPI,education_num,department_num,previous_year_rating_norm
66,Finance,region_22,Bachelor's,m,other,1,27,0.0,1,1,1,58,1,26,58,10,4,2,7,0.0
138,Technology,region_14,Bachelor's,f,other,1,31,4.0,4,1,1,78,0,27,78,9,4,2,3,0.8
140,Technology,region_29,Bachelor's,f,other,2,30,3.0,7,1,1,76,0,23,152,7,4,2,3,0.6
200,Sales & Marketing,region_23,Master's & above,f,sourcing,1,34,4.0,5,1,1,94,1,29,94,9,4,3,1,0.8
230,Operations,region_13,Master's & above,f,other,1,38,5.0,2,1,1,93,1,36,93,12,4,3,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54494,Procurement,region_7,Bachelor's,f,sourcing,1,41,4.0,7,1,1,72,0,34,72,8,4,2,6,0.8
54503,Sales & Marketing,region_25,Master's & above,m,sourcing,2,32,4.0,5,1,1,92,1,27,184,9,4,3,1,0.8
54772,Operations,region_27,Master's & above,m,other,1,37,5.0,2,1,1,57,0,35,57,12,4,3,2,1.0
54797,Sales & Marketing,region_11,Bachelor's,m,sourcing,1,28,5.0,3,1,1,47,0,25,47,12,4,2,1,1.0


In [ ]:
df.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met>80%,awards_won?,avg_training_score,is_promoted,starting_age,total_training_score,age_rating,age_KPI,education_num,department_num,previous_year_rating_norm
0,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0,27,49,12,4,3,1,1.0
1,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0,26,60,12,2,2,2,1.0
2,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0,27,50,7,2,2,1,0.6
3,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0,29,100,2,2,2,1,0.2
4,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0,43,73,5,2,2,3,0.6


In [ ]:
df = df.drop('employee_id',axis = 1)

In [ ]:
for col in df.columns:
  percent = 100 * df[col].isna().sum() / df.shape[0]
  percent = round(percent,2)
  print(f'{col}: {df[col].isna().sum()} ({percent}%) missing values')

department: 0 (0.0%) missing values
region: 0 (0.0%) missing values
education: 2409 (4.4%) missing values
gender: 0 (0.0%) missing values
recruitment_channel: 0 (0.0%) missing values
no_of_trainings: 0 (0.0%) missing values
age: 0 (0.0%) missing values
previous_year_rating: 4124 (7.52%) missing values
length_of_service: 0 (0.0%) missing values
KPIs_met >80%: 0 (0.0%) missing values
awards_won?: 0 (0.0%) missing values
avg_training_score: 0 (0.0%) missing values
is_promoted: 0 (0.0%) missing values


In [ ]:
df['education'].unique()

array(["Master's & above", "Bachelor's", nan, 'Below Secondary'],
      dtype=object)

In [ ]:
#Replacing NaN instead of dropping b/c I want to keep those rows
df['education'] = df.education.fillna('Unknown')

In [ ]:
df['previous_year_rating'].unique()

array([ 5.,  3.,  1.,  4., nan,  2.])

In [ ]:
#Ratings are 1-5, so 0 will mean employee was not here last year
df['previous_year_rating'] = df.previous_year_rating.fillna(0)

In [ ]:
df.dtypes

department               object
region                   object
education                object
gender                   object
recruitment_channel      object
no_of_trainings           int64
age                       int64
previous_year_rating    float64
length_of_service         int64
KPIs_met >80%             int64
awards_won?               int64
avg_training_score        int64
is_promoted               int64
dtype: object

In [ ]:
df.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [ ]:
df['starting_age'] = df['age']-df['length_of_service']
df.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,starting_age
0,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0,27
1,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0,26
2,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0,27
3,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0,29
4,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0,43


In [ ]:
df['total_training_score'] = df['no_of_trainings']*df['avg_training_score']
df.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,starting_age,total_training_score
0,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0,27,49
1,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0,26,60
2,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0,27,50
3,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0,29,100
4,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0,43,73


In [ ]:
df = df.rename(columns={'KPIs_met >80%': 'KPIs_met>80%'})

In [ ]:
#based off of percentages of promotion. Lowered older than 40 with 0 rating bc only 11 people
def age_rating(row):
  if row['age'] <= 40 and row['previous_year_rating'] == 0:
    return 10
  elif row['age'] <= 40 and row['previous_year_rating'] == 1:
    return 2
  elif row['age'] <= 40 and row['previous_year_rating'] == 2:
    return 4
  elif row['age'] <= 40 and row['previous_year_rating'] == 3:
    return 7
  elif row['age'] <= 40 and row['previous_year_rating'] == 4:
    return 9
  elif row['age'] <= 40 and row['previous_year_rating'] == 5:
    return 12
  elif row['age'] > 40 and row['previous_year_rating'] == 0:
    return 6
  elif row['age'] > 40 and row['previous_year_rating'] == 1:
    return 1
  elif row['age'] > 40 and row['previous_year_rating'] == 2:
    return 3
  elif row['age'] > 40 and row['previous_year_rating'] == 3:
    return 5
  elif row['age'] > 40 and row['previous_year_rating'] == 4:
    return 8
  else:
    return 11

In [ ]:
def age_KPI(row):
  if row['age'] <= 45 and row['KPIs_met>80%'] == 0:
    return 2
  elif row['age'] <= 45 and row['KPIs_met>80%'] == 1:
    return 4
  elif row['age'] > 45 and row['KPIs_met>80%'] == 0:
    return 3
  else:
    return 1

In [ ]:
df['age_rating'] = df.apply(age_rating, axis=1)

In [ ]:
df['age_KPI'] = df.apply(age_KPI, axis=1)

In [ ]:
df['education_num'] = df['education'].replace(["Bachelor's","Master's & above","Below Secondary","Unknown"],[2,3,1,0])
df['department_num'] = df['department'].replace(['Sales & Marketing', 'Operations', 'Technology', 'Analytics',
       'R&D', 'Procurement', 'Finance', 'HR', 'Legal'],[1,2,3,4,5,6,7,8,9])
df['previous_year_rating_norm'] = (df['previous_year_rating']-df['previous_year_rating'].min())/(df['previous_year_rating'].max()-df['previous_year_rating'].min())

In [ ]:
df.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met>80%,awards_won?,avg_training_score,is_promoted,starting_age,total_training_score,age_rating,age_KPI,education_num,department_num,previous_year_rating_norm
0,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0,27,49,12,4,3,1,1.0
1,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0,26,60,12,2,2,2,1.0
2,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0,27,50,7,2,2,1,0.6
3,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0,29,100,2,2,2,1,0.2
4,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0,43,73,5,2,2,3,0.6


#Looking for general facts

In [ ]:
#Rating 0 means they were not there last year
#What do these 8% of people do to get promoted?
for i in df['previous_year_rating'].unique():
  rating_df = df[df['previous_year_rating'] == i]
  total = rating_df.shape[0]
  promoted = rating_df[rating_df['is_promoted'] == 1].shape[0]
  percent = 100*promoted/total
  print(f"Employee's with previous year rating {i} were promoted {percent: .2f}% of the time. ({promoted} out of {total})")

Employee's with previous year rating 5.0 were promoted  16.36% of the time. (1921 out of 11741)
Employee's with previous year rating 3.0 were promoted  7.28% of the time. (1355 out of 18618)
Employee's with previous year rating 1.0 were promoted  1.41% of the time. (88 out of 6223)
Employee's with previous year rating 4.0 were promoted  7.94% of the time. (784 out of 9877)
Employee's with previous year rating 0.0 were promoted  8.22% of the time. (339 out of 4124)
Employee's with previous year rating 2.0 were promoted  4.28% of the time. (181 out of 4225)


In [ ]:
zero_df = df[df['previous_year_rating'] == 0]

In [ ]:
#Looks like average training score is an important factor for promotions
#0 is not promoted
#1 is promoted
for i, group in zero_df.groupby('is_promoted'):
  training_score_mean = group['avg_training_score'].mean()
  print(f"The average training score for {i} with a 0 previous year rating was {training_score_mean}")

for i, group in df.groupby('is_promoted'):
  training_score_mean = group['avg_training_score'].mean()
  print(f"The average training score for all people for {i} rating was {training_score_mean}")


The average training score for 0 with a 0 previous year rating was 62.589696169088505
The average training score for 1 with a 0 previous year rating was 71.929203539823
The average training score for all people for 0 rating was 62.64768647786199
The average training score for all people for 1 rating was 71.32519280205655


In [ ]:
#This seems hand picked
for department,group in df.groupby('department'):
  total = group.shape[0]
  promoted = group[group['is_promoted'] == 1].shape[0]
  perecnt = 100*promoted/total
  print(f"Employees in the {department} department were promoted {percent:.2f}% of the time. ({promoted} out of {total})")

Employees in the Analytics department were promoted 4.28% of the time. (512 out of 5352)
Employees in the Finance department were promoted 4.28% of the time. (206 out of 2536)
Employees in the HR department were promoted 4.28% of the time. (136 out of 2418)
Employees in the Legal department were promoted 4.28% of the time. (53 out of 1039)
Employees in the Operations department were promoted 4.28% of the time. (1023 out of 11348)
Employees in the Procurement department were promoted 4.28% of the time. (688 out of 7138)
Employees in the R&D department were promoted 4.28% of the time. (69 out of 999)
Employees in the Sales & Marketing department were promoted 4.28% of the time. (1213 out of 16840)
Employees in the Technology department were promoted 4.28% of the time. (768 out of 7138)


In [ ]:
#These are not as similar which is good
#Seems like females are being promoted more than males
for department,group in df.groupby('department'):
  male_df = group[group['gender'] == 'm']
  female_df = group[group['gender'] == 'f']
  total_male = male_df.shape[0]
  total_female = female_df.shape[0]
  promoted_male = male_df[male_df['is_promoted'] == 1].shape[0]
  promoted_female = female_df[female_df['is_promoted'] == 1].shape[0]
  percent_male = 100*promoted_male/total_male
  percent_female = 100*promoted_female/total_female
  print(f"Males in the {department} department were promoted {percent_male:.2f}% of the time. ({promoted_male} out of {total_male})")
  print(f"Females in the {department} department were promoted {percent_female:.2f}% of the time. ({promoted_female} out of {total_female})")
  print('='*100)

Males in the Analytics department were promoted 9.30% of the time. (450 out of 4839)
Females in the Analytics department were promoted 12.09% of the time. (62 out of 513)
Males in the Finance department were promoted 8.14% of the time. (151 out of 1855)
Females in the Finance department were promoted 8.08% of the time. (55 out of 681)
Males in the HR department were promoted 5.52% of the time. (78 out of 1412)
Females in the HR department were promoted 5.77% of the time. (58 out of 1006)
Males in the Legal department were promoted 5.28% of the time. (47 out of 890)
Females in the Legal department were promoted 4.03% of the time. (6 out of 149)
Males in the Operations department were promoted 8.71% of the time. (581 out of 6671)
Females in the Operations department were promoted 9.45% of the time. (442 out of 4677)
Males in the Procurement department were promoted 7.87% of the time. (303 out of 3851)
Females in the Procurement department were promoted 11.71% of the time. (385 out of 328